In [21]:
import tensorflow as tf 
import numpy as np
import torch
import math
import os
import pandas as pd
import matplotlib.pyplot as plt 
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score

In [3]:
from sklearn.cluster import KMeans


In [428]:
#OP Model with L1 norm
from tensorflow.python.ops.linalg_ops import norm_v2
class OP_main():
    
    def __init__(self,Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25):
        
        #(Data, lam,th, rho, mup)=value_sets


        n1,n2=np.shape(Data)
        self.lam=lam #1/np.sqrt(n1)
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0]#norm_v2(Data,2)
        #print(norm_two)
        norm_inf=norm_v2(Data,np.inf)/self.lam
        #print(norm_inf)
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm
        #print(Lambda_int[0,0]) 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=200 
        self.tol=10**(-7) 
        
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])
        #print(self.E[-1][0,0])
        #print(self.X[-1][0,0])  
        #print(self.Lambda[-1][0,0])  
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion))
            #print(self.E[-1][0,0])
            #print(self.X[-1][0,0]) 
            #print(self.Lambda[-1][0,0])
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda):
        
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=tf.nn.relu(S_int-tf.multiply(th,tf.ones(np.shape(S_int), tf.float64)))
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
      
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(tf.nn.relu(tf.norm(E[:,i])-tau)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         


In [432]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=1.3 #1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
H=OP_main(Data,lam,th,rho,mup)
print(H.Elabel)
print(H.FE)

Stop Criterion: 0.02771651236316883
Stop Criterion: 0.017384501214288788
Stop Criterion: 0.004953136813105215
Stop Criterion: 0.0012521199148616802
Stop Criterion: 0.0002414259085647203
Stop Criterion: 3.704678373148938e-05
Stop Criterion: 7.225579794613062e-06
Stop Criterion: 2.2373322264342416e-06
Stop Criterion: 8.001266062565949e-07
Stop Criterion: 3.0496679119347266e-07
Stop Criterion: 1.2169189011155522e-07
Stop Criterion: 5.02008332645738e-08
[1, 0, 0]
tf.Tensor(
[[ 0.79596435  0.          0.        ]
 [ 0.4636718   0.          0.        ]
 [-0.24470637  0.          0.        ]], shape=(3, 3), dtype=float64)


In [433]:
#OP model with GST
from tensorflow.python.ops.linalg_ops import norm_v2
class OPGST_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1):
        
        #(Data, lam,th, rho, mup)=value_sets

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam #1/np.sqrt(n1)
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0]#norm_v2(Data,2)
        #print(norm_two)
        norm_inf=norm_v2(Data,np.inf)/self.lam
        #print(norm_inf)
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm
        #print(Lambda_int[0,0]) 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=200 
        self.tol=10**(-7) 
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])
        #print(self.E[-1][0,0])
        #print(self.X[-1][0,0])  
        #print(self.Lambda[-1][0,0])  
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion))
            #print(self.E[-1][0,0])
            #print(self.X[-1][0,0]) 
            #print(self.Lambda[-1][0,0])
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda):#需要修改
      
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.Generalized_Soft_Thresholding(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
        
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.Generalized_Soft_Thresholding(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         
        
    def Generalized_Soft_Thresholding(self,S,weight,p):
        #diagS: a vector
        #weight: a value
        if np.shape(S) == ():
            diagS=[S]
        else:
            diagS=S#np.array(diagS,np.float64)
        J=5 
        sigma0   =    np.abs(diagS)
        tau_GST  =   (2*weight*(1-p))**(1/(2-p))   +   p*weight*(2*(1-p)*weight)**((p-1)/(2-p)) 
        Delta=[];
        for i in range(len(diagS)):
                if sigma0[i]>tau_GST: 
                    delta=sigma0[i]  
                    for k in range(J): 
                            delta=sigma0[i]-weight*p*delta**(p-1)           
                            k=k+1  
                    Delta.append(np.sign(diagS[i])*delta)  
                else:
                    Delta.append(0) 
        if np.shape(S) == ():
            return np.array(Delta[0],np.float64) 
        else:
            return np.array(Delta,np.float64) 


    
 

In [434]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=1.1 #1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=0.8
#value_sets=(Data,lam,th,rho,mup,p)
HGST=OPGST_main(Data,lam,th,rho,mup,p)
print(HGST.Elabel)
print(HGST.FE)

Stop Criterion: 0.06968655348307115
Stop Criterion: 0.033549524337254284
Stop Criterion: 0.014605080229469644
Stop Criterion: 0.0064766240023816065
Stop Criterion: 0.003569636981640282
Stop Criterion: 0.0013102399929058853
Stop Criterion: 0.00015949477959337335
Stop Criterion: 4.1360014467796234e-05
Stop Criterion: 1.8868610062230848e-05
Stop Criterion: 5.632345409268264e-06
Stop Criterion: 2.2922222900111563e-06
Stop Criterion: 1.0333381822169081e-06
Stop Criterion: 4.706433338958856e-07
Stop Criterion: 2.0817622293720724e-07
Stop Criterion: 9.149778874305399e-08
[1, 0, 0]
tf.Tensor(
[[ 6.38093858 -0.          0.        ]
 [ 3.76187763  0.          0.        ]
 [-1.85718315  0.          0.        ]], shape=(3, 3), dtype=float64)


In [435]:
#OP model with ETP
from tensorflow.python.ops.linalg_ops import norm_v2
class OPETP_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1,Maxiter=1000,tol=10**(-7)):
        
        #(Data, lam,th, rho, mup)=value_sets

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam #1/np.sqrt(n1)
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0]#norm_v2(Data,2)
        #print(norm_two)
        norm_inf=norm_v2(Data,np.inf)/self.lam
        #print(norm_inf)
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm
        #print(Lambda_int[0,0]) 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=200
        self.tol=10**(-7) 
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])
        #print(self.E[-1][0,0])
        #print(self.X[-1][0,0])  
        #print(self.Lambda[-1][0,0])  
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion))
            #print(self.E[-1][0,0])
           # print(self.X[-1][0,0]) 
            #print(self.Lambda[-1][0,0])
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda): 
      
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.GAI_ETP(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda): 
        Temp= Data-X+Lambda/self.mu 
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.GAI_ETP(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         
        
   

    def GAI_ETP(self,xi,lambd,gamma,Iter=10,tol=10**(-6)):
        #Iter=10; 
        a0=np.max([np.log((1-np.exp(-gamma))/(lambd*gamma**2))/(-gamma),0])
        #print(a0)
        delta=a0+self.ETP_gradient(a0,gamma,lambd)
        #print(delta) 
        if np.shape(xi) == ():
            x=[xi]
        else:
            x=xi
        n=len(x)
        y=[]
 

     
        for j in range(n):
            gradient_g= self.ETP_gradient(x[j],gamma,lambd)
            if gradient_g==0:
                BARx_b=x[j] 
            else:
                if delta<x[j]:
                    a=x[j]
                    sto=0
                    kkk=0
                    while sto==0:
                        kkk=kkk+1
                        a1=x[j]-self.ETP_gradient(a,gamma,lambd)
                        a2=x[j]-self.ETP_gradient(a1,gamma,lambd) 
                        if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
                            if np.abs(a2-2*a1+a)<self.eps:
                                BARx_b=a2
                            else:
                                BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
                            break
                        a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a) 
                        #print(a)
                else:
                    BARx_b=a0
            if self.ETP_fun(BARx_b,x[j],gamma,lambd)<=self.ETP_fun(0,x[j],gamma,lambd):
                y.append(BARx_b)   
            else:
                y.append(0)
                    
        if np.shape(xi) == ():
            return np.array(y[0],np.float64) 
        else:
            return np.array(y,np.float64)
      
           
    #ETP's gradient 
    def ETP_gradient(self,x,gamma,lambd):
        gradient_g=gamma*lambd*np.exp(-gamma*x)/(1-np.exp(-gamma)) 
        return gradient_g
 
    def ETP_fun(self,x,y,gamma,lambd):
        g=1/2*(y-x)**2+lambd*(1-np.exp(-gamma*x))/(1-np.exp(-gamma))
        return g   



In [436]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=6#1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=10
#value_sets=(Data,lam,th,rho,mup,p)
HETP=OPETP_main(Data,lam,th,rho,mup,p)
print(HETP.Elabel)
print(HETP.FE)

Stop Criterion: 2.4261219471294536e-16
[1, 0, 0]
tf.Tensor(
[[ 7.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [ 5.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [-5.92118946e-15 -0.00000000e+00  0.00000000e+00]], shape=(3, 3), dtype=float64)


In [437]:
# eps=0.000000000000001
# def sparse_col(E, tau,p): 
#         result=[]
#         n1,n2=np.shape(E)
#         for i in range(n2):
#             print(tf.norm(E[:,i]))
#             print(GAI_ETP(tf.norm(E[:,i]),tau,p))
#             result.append(GAI_ETP(tf.norm(E[:,i]),tau,p)*E[:,i]/(tf.norm(E[:,i])+eps)) 
#         return tf.transpose(result,[1,0])

# def GAI_ETP(xi,lambd,gamma,Iter=10,tol=10**(-6)):
#         #Iter=10; 
#         a0=np.max([np.log((1-np.exp(-gamma))/(lambd*gamma**2))/(-gamma),0])
#         #print(a0)
#         delta=a0+ETP_gradient(a0,gamma,lambd)
#         #print(delta) 
#         if np.shape(xi) == ():
#             x=[xi]
#         else:
#             x=xi
#         n=len(x)
#         y=[]
 

     
#         for j in range(n):
#             gradient_g= ETP_gradient(x[j],gamma,lambd)
#             if gradient_g==0:
#                 BARx_b=x[j] 
#             else:
#                 if delta<x[j]:
#                     a=x[j]
#                     sto=0
#                     kkk=0
#                     while sto==0:
#                         kkk=kkk+1
#                         a1=x[j]-ETP_gradient(a,gamma,lambd)
#                         a2=x[j]-ETP_gradient(a1,gamma,lambd) 
#                         if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
#                             if a2-2*a1+a<eps:
#                                 BARx_b=a2
#                             else:
#                                 BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
#                             break
#                         a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a) 
#                         #print(a)
#                 else:
#                     BARx_b=a0
#             if ETP_fun(BARx_b,x[j],gamma,lambd)<=ETP_fun(0,x[j],gamma,lambd):
#                 y.append(BARx_b)   
#             else:
#                 y.append(0)
                    
#         if np.shape(xi) == ():
#             return np.array(y[0],np.float64) 
#         else:
#             return np.array(y,np.float64)
      
           
#     #ETP's gradient 
# def ETP_gradient(x,gamma,lambd):
#     gradient_g=gamma*lambd*np.exp(-gamma*x)/(1-np.exp(-gamma)) 
#     return gradient_g
 
# def ETP_fun(x,y,gamma,lambd):
#     g=1/2*(y-x)**2+lambd*(1-np.exp(-gamma*x))/(1-np.exp(-gamma))
#     return g   

# print(sparse_col(np.array([[12.6,3.6,5.4],[9.0,9.0,10.8],[0,14.4,16.2]],np.float64),13.7196,0.8))

In [439]:
# OP Model with LOG
from tensorflow.python.ops.linalg_ops import norm_v2
class OPLOG_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1,Maxiter=1000,tol=10**(-7) ):
        
        #(Data, lam,th, rho, mup)=value_sets

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam #1/np.sqrt(n1)
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0]#norm_v2(Data,2)
        #print(norm_two)
        norm_inf=norm_v2(Data,np.inf)/self.lam
        #print(norm_inf)
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm
        #print(Lambda_int[0,0]) 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=Maxiter
        self.tol=tol
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])
        #print(self.E[-1][0,0])
        #print(self.X[-1][0,0])  
        #print(self.Lambda[-1][0,0])  
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion))
            #print(self.E[-1][0,0])
            #print(self.X[-1][0,0]) 
            #print(self.Lambda[-1][0,0])
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda): 
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.GAI_LOG(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
        
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.GAI_LOG(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         

    def LOG_th(self,x,tau,lambd):
        x0=np.sqrt(2*lambd)-tau
        y=np.sign(np.max([x-x0,0]))*(1/2)*((x-tau)+np.sqrt((x+tau)**2-2*lambd))+np.sign(np.max([-x0-x,0]))*(1/2)*((x+tau)+np.sqrt((x-tau)**2-2*lambd))
        return y
    
    
    
    def GAI_LOG(self,xi,lambd,gamma,Iter=10,tol=10**(-4)):
        #Iter=10; 
    
        a0=np.max([np.sqrt(lambd)-gamma,0])
        #print(a0)
        delta=a0+self.LOG_gradient(a0,gamma,lambd)
        #print(delta) 
        if np.shape(xi) == ():
            x=[xi]
        else:
            x=xi
        n=len(x)
        y=[] 
     
        for j in range(n):
            gradient_g= self.LOG_gradient(x[j],gamma,lambd)
            if gradient_g==0:
                BARx_b=x[j] 
            else:
                if delta<x[j]:
                    a=x[j]
                    sto=0
                    kkk=0
                    while sto==0:
                        kkk=kkk+1
                        a1=x[j]-self.LOG_gradient(a,gamma,lambd)
                        a2=x[j]-self.LOG_gradient(a1,gamma,lambd) 
                        if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
                            if np.abs(a2-2*a1+a)<self.eps:
                                BARx_b=a2
                            else:
                                BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
                            break
                        a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a) 
                        #print(a)
                else:
                    BARx_b=a0
                if self.LOG_fun(BARx_b,x[j],gamma,lambd)<=self.LOG_fun(0,x[j],gamma,lambd):
                    y.append(BARx_b)   
                else:
                    y.append(0)
        
        if np.shape(xi) == ():
            return np.array(y[0],np.float64) 
        else:
            return np.array(y,np.float64)
      
           


#LOG's gradient 
    def LOG_gradient(self,x,gamma,lambd):
        gradient_g=lambd/(gamma+x)
        return gradient_g
 
    def LOG_fun(self,x,y,gamma,lambd):
        g=1/2*(y-x)**2+lambd*(np.log(gamma+x))
        return g
 

In [444]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=9#1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=23
#value_sets=(Data,lam,th,rho,mup,p)
HLOG=OPLOG_main(Data,lam,th,rho,mup,p)
print(HLOG.Elabel)
print(HLOG.FE)

Stop Criterion: 0.0026366742250366285
Stop Criterion: 0.0016644579340530348
Stop Criterion: 0.000599125718678935
Stop Criterion: 0.0002322995954917029
Stop Criterion: 9.510577059916805e-05
Stop Criterion: 4.023698178748473e-05
Stop Criterion: 1.736866056257894e-05
Stop Criterion: 7.594570802103793e-06
Stop Criterion: 3.351016147898213e-06
Stop Criterion: 1.4909650435274126e-06
Stop Criterion: 4.1122749716615434e-07
Stop Criterion: 1.6144436485787669e-07
Stop Criterion: 7.165158124760496e-08
[1, 0, 0]
tf.Tensor(
[[ 1.51067052 -0.         -0.        ]
 [ 1.03186057  0.          0.        ]
 [-0.11010954  0.          0.        ]], shape=(3, 3), dtype=float64)


In [357]:
# eps=0.000000000000001
# def sparse_col(E, tau,p): 
#         result=[]
#         n1,n2=np.shape(E)
#         for i in range(n2):
#             result.append(GAI_LOG(tf.norm(E[:,i]),tau,p)*E[:,i]/(tf.norm(E[:,i])+eps)) 
#             print(E[:,i])
#             print((tf.norm(E[:,i]))
#         return tf.transpose(result,[1,0])
    
    
# def GAI_LOG(xi,lambd,gamma,Iter=10,tol=10**(-4)):
#         #Iter=10; 
    
#         a0=np.max([np.sqrt(lambd)-gamma,0])
#         #print(a0)
#         delta=a0+LOG_gradient(a0,gamma,lambd)
#         #print(delta) 
#         if np.shape(xi) == ():
#             x=[xi]
#         else:
#             x=xi
#         n=len(x)
#         y=[]
 

     
#         for j in range(n):
#             gradient_g= LOG_gradient(x[j],gamma,lambd)
#             if gradient_g==0:
#                 BARx_b=x[j] 
#             else:
#                 if delta<x[j]:
#                     a=x[j]
#                     sto=0
#                     kkk=0
#                     while sto==0:
#                         kkk=kkk+1
#                         a1=x[j]-LOG_gradient(a,gamma,lambd)
#                         a2=x[j]-LOG_gradient(a1,gamma,lambd) 
#                         if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
#                             BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
#                             break
#                         a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a) 
#                         #print(a)
#                 else:
#                     BARx_b=a0
#             if LOG_fun(BARx_b,x[j],gamma,lambd)<=LOG_fun(0,x[j],gamma,lambd):
#                  y.append(BARx_b)   
#             else:
#                  y.append(0)
        
#         if np.shape(xi) == ():
#             return np.array(y[0],np.float64) 
#         else:
#             return np.array(y,np.float64)
      
          






# #LOG's gradient 
# def LOG_gradient(x,gamma,lambd):
#         gradient_g=lambd/(gamma+x)
#         return gradient_g
    
# def LOG_fun(x,y,gamma,lambd):
#         g=1/2*(y-x)**2+lambd*(np.log(gamma+x))
#         return g

# print(sparse_col(np.array([[7],[5],[0]],np.float64),34,56))

# German Credit Data

In [68]:
#loading data Geman  
df = pd.read_csv('german.csv')
df.tail(3)
print(df['Creditability;Account_Balance;Duration_of_Credit_monthly;Payment_Status_of_Previous_Credit;Purpose;Credit_Amount;Value_Savings_Stocks;Length_of_current_employment;Instalment_per_cent;Sex_Marital_Status;Guarantors;Duration_in_Current_address;Most_valuable_available_asset;Age_years;Concurrent_Credits;Type_of_apartment;No_of_Credits_at_this_Bank;Occupation;No_of_dependents;Telephone;Foreign_Worker'])
name='Creditability;Account_Balance;Duration_of_Credit_monthly;Payment_Status_of_Previous_Credit;Purpose;Credit_Amount;Value_Savings_Stocks;Length_of_current_employment;Instalment_per_cent;Sex_Marital_Status;Guarantors;Duration_in_Current_address;Most_valuable_available_asset;Age_years;Concurrent_Credits;Type_of_apartment;No_of_Credits_at_this_Bank;Occupation;No_of_dependents;Telephone;Foreign_Worker'

0       1;1;18;4;2;1049;1;2;4;2;1;4;2;21;3;1;1;3;1;1;1
1        1;1;9;4;0;2799;1;3;2;3;1;2;1;36;3;1;2;3;2;1;1
2        1;2;12;2;9;841;2;4;2;2;1;4;1;23;3;1;1;2;1;1;1
3       1;1;12;4;0;2122;1;3;3;3;1;2;1;39;3;1;2;2;2;1;2
4       1;1;12;4;0;2171;1;3;4;3;1;4;2;38;1;2;2;2;1;1;2
                            ...                       
995     0;1;24;2;3;1987;1;3;2;3;1;4;1;21;3;1;1;2;2;1;1
996     0;1;24;2;0;2303;1;5;4;3;2;1;1;45;3;2;1;3;1;1;1
997    0;4;21;4;0;12680;5;5;4;3;1;4;4;30;3;3;1;4;1;2;1
998     0;2;12;2;3;6468;5;1;2;3;1;1;4;52;3;2;1;4;1;2;1
999     0;1;30;2;2;6350;5;5;4;3;1;4;2;31;3;2;1;3;1;1;1
Name: Creditability;Account_Balance;Duration_of_Credit_monthly;Payment_Status_of_Previous_Credit;Purpose;Credit_Amount;Value_Savings_Stocks;Length_of_current_employment;Instalment_per_cent;Sex_Marital_Status;Guarantors;Duration_in_Current_address;Most_valuable_available_asset;Age_years;Concurrent_Credits;Type_of_apartment;No_of_Credits_at_this_Bank;Occupation;No_of_dependents;Telephone;Foreig

In [74]:
# data processing
def gdata(df):
    data=[]
    for i in range(len(df[name])):
        rowv=[]
        ss=''
        for x in range(len(df[name][i])):
            #rowv=[]
            if df[name][i][x]!=';':
                ss += df[name][i][x]
                #print(ss)
            else:
                rowv.append(int(ss))
                ss=''
        rowv.append(int(ss))
        data.append(rowv)
    return data

def gtit(name):
    tit=[]
    ss=''
    for x in range(len(name)):
        #rowv=[]
        if name[x]!=';':
            ss += name[x]
            #print(ss)
        else:
            tit.append(ss)
            ss=''
    tit.append(ss)
    return tit

print(name)
titg=gtit(name)
for x in range(len(titg)):
    print(titg[x])
print(titg)
    


datag=gdata(df)
print(df[name][919])
print(datag[919])


Tlabel=tf.ones(np.shape(tf.transpose(datag,[1,0])[0,:]))-np.array(tf.transpose(datag,[1,0])[0,:],np.float64)
retio_intrinsic=1-np.sum(Tlabel)/len(Tlabel)
print(Tlabel)
print("retio of intrinsic samples: {}".format(retio_intrinsic)) 
Data=np.array(tf.transpose(datag,[1,0]),np.float64)[1:,:] 
print(np.shape(Data))

Creditability;Account_Balance;Duration_of_Credit_monthly;Payment_Status_of_Previous_Credit;Purpose;Credit_Amount;Value_Savings_Stocks;Length_of_current_employment;Instalment_per_cent;Sex_Marital_Status;Guarantors;Duration_in_Current_address;Most_valuable_available_asset;Age_years;Concurrent_Credits;Type_of_apartment;No_of_Credits_at_this_Bank;Occupation;No_of_dependents;Telephone;Foreign_Worker
Creditability
Account_Balance
Duration_of_Credit_monthly
Payment_Status_of_Previous_Credit
Purpose
Credit_Amount
Value_Savings_Stocks
Length_of_current_employment
Instalment_per_cent
Sex_Marital_Status
Guarantors
Duration_in_Current_address
Most_valuable_available_asset
Age_years
Concurrent_Credits
Type_of_apartment
No_of_Credits_at_this_Bank
Occupation
No_of_dependents
Telephone
Foreign_Worker
['Creditability', 'Account_Balance', 'Duration_of_Credit_monthly', 'Payment_Status_of_Previous_Credit', 'Purpose', 'Credit_Amount', 'Value_Savings_Stocks', 'Length_of_current_employment', 'Instalment_per_

In [132]:
# test OP
n1,n2=np.shape(Data)
print([n1,n2])
lam=5.5/np.sqrt(n2)
print(lam*np.sqrt(n2))
th=0.0000000000000001
rho=1.5
mup=1.25
value_sets=(Data, lam, th,rho, mup)
H=OP_main(value_sets) 
#print(Tlabel)
print(H.Elabel)
acc_op=accuracy_score(Tlabel, H.Elabel)
print("acc by OP: {}".format(acc_op)) 
precision_op=precision_score(Tlabel, H.Elabel)
print("Precision by OP: {}".format(precision_op)) 
rs_op=recall_score(Tlabel, H.Elabel)
print("recall score by OP: {}".format(rs_op))  
f1_op=f1_score(Tlabel, H.Elabel)
print("f1 score by OP: {}".format(f1_op))
r2_op=r2_score(Tlabel, H.Elabel)
print("r2 score by OP: {}".format(r2_op))

[20, 1000]
Stop Criterion: 0.04172300635169311
Stop Criterion: 0.01362955957421532
Stop Criterion: 0.0073098712129353914
Stop Criterion: 0.0068196930042786034
Stop Criterion: 0.006810130460883333
Stop Criterion: 0.006810042752252523
Stop Criterion: 0.006810042368178341
Stop Criterion: 0.006810042367375473
Stop Criterion: 0.006810042367374643
Stop Criterion: 0.002903295734016509
Stop Criterion: 0.0025361118864823907
Stop Criterion: 0.0019905209335913283
Stop Criterion: 0.001139340378695178
Stop Criterion: 0.0011261994345431899
Stop Criterion: 0.0010597565526406672
Stop Criterion: 0.0007714559865651052
Stop Criterion: 0.0006544382631864233
Stop Criterion: 0.00038489810074208693
Stop Criterion: 0.00023576825307294782
Stop Criterion: 0.00011818770840758773
Stop Criterion: 3.7803857740678286e-05
Stop Criterion: 2.5493437678640737e-05
Stop Criterion: 1.1873797279431136e-05
Stop Criterion: 3.884116318610258e-06
Stop Criterion: 6.689151540769852e-07
Stop Criterion: 4.5566906857802e-07
Stop Cri

In [89]:
print(H.Elabel)

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 

In [83]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(tf.transpose(Data,[1,0])) 
y_pred = kmeans.predict(tf.transpose(Data,[1,0]))
print(y_pred)
finalacc_kM=accuracy_score(Tlabel, y_pred)
print("acc by kM: {}".format(finalacc_kM)) 
precision_kM=precision_score(Tlabel, y_pred)
print("Precision by kM: {}".format(precision_kM)) 
rs_kM=recall_score(Tlabel, y_pred)
print("recall score by kM: {}".format(rs_kM))  
f1_kM=f1_score(Tlabel, y_pred)
print("f1 score by kM: {}".format(f1_kM))
r2_kM=r2_score(Tlabel, y_pred)
print("r2 score by kM: {}".format(r2_kM))
#score.append(silhouette_score(features, kmeans.labels_))

[0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 

In [86]:
#print(tf.ones(len(y_pred))-y_pred)
#acc_knn=accuracy_score(Tlabel, tf.ones(len(y_pred))-y_pred)
#print("acc by KNN: {}".format(acc_knn)) 
#precision_knn=precision_score(Tlabel, tf.ones(len(y_pred))-y_pred)
#print("Precision by KNN: {}".format(precision_knn)) 
#rs_knn=recall_score(Tlabel, tf.ones(len(y_pred))-y_pred)
#print("recall score by KNN: {}".format(rs_knn))  
#f1_knn=f1_score(Tlabel, tf.ones(len(y_pred))-y_pred)
#print("f1 score by KNN: {}".format(f1_knn))
#r2_knn=r2_score(Tlabel, tf.ones(len(y_pred))-y_pred)
#print("r2 score by KNN: {}".format(r2_knn))

In [91]:
from sklearn.mixture import GaussianMixture
gm=GaussianMixture(n_components=2,n_init=10)
gm.fit(tf.transpose(Data,[1,0]))
gm.converged_
gm.n_iter_
y_gm=gm.predict(tf.transpose(Data,[1,0]))
#gm.predict_proba(tf.transpose(Data,[1,0]))
print(y_gm)
finalacc_gm=accuracy_score(Tlabel, tf.ones(len(y_gm))-y_gm)
print("acc by Gaussian Mixture: {}".format(finalacc_gm)) 
precision_gm=precision_score(Tlabel, tf.ones(len(y_gm))-y_gm)
print("Precision by Gaussian Mixture: {}".format(precision_gm)) 
rs_gm=recall_score(Tlabel, tf.ones(len(y_gm))-y_gm)
print("recall score by Gaussian Mixture: {}".format(rs_gm))  
f1_gm=f1_score(Tlabel, tf.ones(len(y_gm))-y_gm)
print("f1 score by Gaussian Mixture: {}".format(f1_gm))
r2_gm=r2_score(Tlabel, tf.ones(len(y_gm))-y_gm)
print("r2 score by Gaussian Mixture: {}".format(r2_gm))

[0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1 0 0 0 0
 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 

In [ ]:
from sklearn.mixture import GaussianMixture
gm=GaussianMixture(n_components=2,n_init=10)
gm.fit(tf.transpose(Data,[1,0]))
gm.converged_
gm.n_iter_
y_gm=gm.predict(tf.transpose(Data,[1,0]))
#gm.predict_proba(tf.transpose(Data,[1,0]))
print(y_gm)
finalacc_gm=accuracy_score(Tlabel, y_gm)
print("acc by Gaussian Mixture: {}".format(finalacc_gm)) 
precision_gm=precision_score(Tlabel, y_gm)
print("Precision by Gaussian Mixture: {}".format(precision_gm)) 
rs_gm=recall_score(Tlabel, y_gm)
print("recall score by Gaussian Mixture: {}".format(rs_gm))  
f1_gm=f1_score(Tlabel, y_gm)
print("f1 score by Gaussian Mixture: {}".format(f1_gm))
r2_gm=r2_score(Tlabel, y_gm)
print("r2 score by Gaussian Mixture: {}".format(r2_gm))

# The Insurance Company (TIC) Benchmark

In [94]:
#loading TIC
df_label = pd.read_csv('tic_2000_target_data.csv')
df_label.head(3)
TlabelTIC=np.array(df_label)[:,0]
print(len(TlabelTIC)) 
df_Data = pd.read_csv('tic_2000_eval_data.csv')
df_Data.head(3)
DataTIC=tf.transpose(np.array(df_Data,np.float64),[1,0])
print(np.shape(DataTIC))
retio_intrinsic=1-np.sum(TlabelTIC)/len(TlabelTIC)
print("retio of intrinsic samples: {}".format(retio_intrinsic)) 

4000
(85, 4000)
retio of intrinsic samples: 0.9405


In [200]:
# test OP
n1,n2=np.shape(DataTIC)
print([n1,n2])
lam=5/np.sqrt(n2)
th=0.0000000000000001
rho=1.5
mup=1.25
value_sets=(DataTIC, lam, th,rho, mup)
H=OP_main(value_sets) 
print(TlabelTIC)
print(H.Elabel)
acc_op=accuracy_score(TlabelTIC, H.Elabel)
print("acc by OP: {}".format(acc_op)) 
precision_op=precision_score(TlabelTIC, H.Elabel)
print("Precision by OP: {}".format(precision_op)) 
rs_op=recall_score(TlabelTIC, H.Elabel)
print("recall score by OP: {}".format(rs_op))  
f1_op=f1_score(TlabelTIC, H.Elabel)
print("f1 score by OP: {}".format(f1_op))
r2_op=r2_score(TlabelTIC, H.Elabel)
print("r2 score by OP: {}".format(r2_op))

[85, 4000]
Stop Criterion: 0.364498481840813
Stop Criterion: 0.2911324512061364
Stop Criterion: 0.25060778515269144
Stop Criterion: 0.1582460955986047
Stop Criterion: 0.07336364437181367
Stop Criterion: 0.030316496556801985
Stop Criterion: 0.008174521466537908
Stop Criterion: 0.001601751821392138
Stop Criterion: 0.0005918295824999644
Stop Criterion: 0.0003286994561150469


KeyboardInterrupt: 

In [63]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(tf.transpose(DataTIC,[1,0])) 
y_pred = kmeans.predict(tf.transpose(DataTIC,[1,0]))
acc_knn=accuracy_score(TlabelTIC, y_pred)
print("acc by KNN: {}".format(acc_knn)) 
precision_knn=precision_score(TlabelTIC, y_pred)
print("Precision by KNN: {}".format(precision_knn)) 
rs_knn=recall_score(TlabelTIC, y_pred)
print("recall score by KNN: {}".format(rs_knn))  
f1_knn=f1_score(TlabelTIC, y_pred)
print("f1 score by KNN: {}".format(f1_knn))
r2_knn=r2_score(TlabelTIC, y_pred)
print("r2 score by KNN: {}".format(r2_knn))

acc by KNN: 0.65075
Precision by KNN: 0.0821917808219178
recall score by KNN: 0.4789915966386555
f1 score by KNN: 0.14030769230769233
r2 score by KNN: -5.24109292839943


In [61]:
#print(tf.ones(len(y_pred))-y_pred)
#acc_knn=accuracy_score(TlabelTIC, tf.ones(len(y_pred))-y_pred)
#print("acc by KNN: {}".format(acc_knn)) 
#precision_knn=precision_score(TlabelTIC, tf.ones(len(y_pred))-y_pred)
#print("Precision by KNN: {}".format(precision_knn)) 
#rs_knn=recall_score(TlabelTIC, tf.ones(len(y_pred))-y_pred)
#print("recall score by KNN: {}".format(rs_knn))  
#f1_knn=f1_score(TlabelTIC, tf.ones(len(y_pred))-y_pred)
#print("f1 score by KNN: {}".format(f1_knn))
#r2_knn=r2_score(TlabelTIC, tf.ones(len(y_pred))-y_pred)
#print("r2 score by KNN: {}".format(r2_knn))

tf.Tensor([1. 0. 1. ... 1. 1. 0.], shape=(4000,), dtype=float32)
acc by KNN: 0.34925
Precision by KNN: 0.047455032529659394
recall score by KNN: 0.5210084033613446
f1 score by KNN: 0.08698702209750964
r2 score by KNN: -10.628893981835159


In [101]:
#from sklearn.mixture import GaussianMixture
#gm=GaussianMixture(n_components=2,n_init=10)
#gm.fit(tf.transpose(DataTIC,[1,0]))
#gm.converged_
#gm.n_iter_
#y_gm=gm.predict(tf.transpose(DataTIC,[1,0]))
##gm.predict_proba(tf.transpose(Data,[1,0]))
#print(y_gm)
#finalacc_gm=accuracy_score(TlabelTIC, tf.ones(len(y_gm))-y_gm)
#print("acc by Gaussian Mixture: {}".format(finalacc_gm)) 
#precision_gm=precision_score(TlabelTIC, tf.ones(len(y_gm))-y_gm)
#print("Precision by Gaussian Mixture: {}".format(precision_gm)) 
#rs_gm=recall_score(TlabelTIC, tf.ones(len(y_gm))-y_gm)
#print("recall score by Gaussian Mixture: {}".format(rs_gm))  
#f1_gm=f1_score(TlabelTIC, tf.ones(len(y_gm))-y_gm)
#print("f1 score by Gaussian Mixture: {}".format(f1_gm))
#r2_gm=r2_score(TlabelTIC, tf.ones(len(y_gm))-y_gm)
#print("r2 score by Gaussian Mixture: {}".format(r2_gm))

In [100]:
from sklearn.mixture import GaussianMixture
gm=GaussianMixture(n_components=2,n_init=10)
gm.fit(tf.transpose(DataTIC,[1,0]))
gm.converged_
gm.n_iter_
y_gm=gm.predict(tf.transpose(DataTIC,[1,0]))
#gm.predict_proba(tf.transpose(Data,[1,0]))
print(y_gm)
finalacc_gm=accuracy_score(TlabelTIC,y_gm)
print("acc by Gaussian Mixture: {}".format(finalacc_gm)) 
precision_gm=precision_score(TlabelTIC, y_gm)
print("Precision by Gaussian Mixture: {}".format(precision_gm)) 
rs_gm=recall_score(TlabelTIC, y_gm)
print("recall score by Gaussian Mixture: {}".format(rs_gm))  
f1_gm=f1_score(TlabelTIC, y_gm)
print("f1 score by Gaussian Mixture: {}".format(f1_gm))
r2_gm=r2_score(TlabelTIC, y_gm)
print("r2 score by Gaussian Mixture: {}".format(r2_gm))

[1 1 1 ... 1 1 1]
acc by Gaussian Mixture: 0.127
Precision by Gaussian Mixture: 0.05955603681645912
recall score by Gaussian Mixture: 0.9243697478991597
f1 score by Gaussian Mixture: 0.1119023397761953
r2 score by Gaussian Mixture: -14.600498572634796
